In [21]:
from xetrack import Reader
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import plotly.io as pio
# pio.renderers.default='notebook'
WORKFLOW = 'append' # random, taxi
df = Reader('../output/stats.db').to_df()
# Filter last experiemnt
df = df[df['track_id']==df.tail(1)['track_id'].iloc[0]]
df = df[df['tech'] != 'shutil']
df = df[df['tech'] != 'm1']
df['name'] = df['name'].str.replace('_',' ')
df['function'] = df['function'].str.replace('_',' ')
df['function'] = df['function'].str.replace(' new upload','').str.replace(' merged upload','')
df['mb/s'] = df['file_bytes']/df['time']
print(f"Errors: {set(df['error'].fillna('').values)}")
print(f"steps count: {df['step'].max()+1}")
print(f"All data size: {len(df)}")
df.tail(2)

Errors: {''}
steps count: 2
All data size: 13


,timestamp,track_id,merged,step,filename,tech,p_memory_percent,memory_percent,cpu,function,bytes_sent,bytes_recv,disk_percent,name,time,error,file_bytes,is_merged,workflow,mb/s
28,17-08-2023 14:48:15.644,39e61e16-af8b-4c1d-aa50-085250981858,false,1,random/data.parquet,xethub,8.527368,246.937486,3.8,pyxet,1.437500,0.265625,13.2,pyxet new upload,21.103928,,1.231064,NaN,random,0.058333
29,17-08-2023 14:48:22.927,39e61e16-af8b-4c1d-aa50-085250981858,false,1,random/data.parquet,lakefs,3.118742,89.777964,0.0,lakefs,2.551758,1.304688,4.8,lakefs new upload,7.221206,,1.231064,NaN,random,0.170479


In [22]:
fig = px.bar(df, x='function', y='mb/s', color='tech')
fig.show()

In [3]:
groups = df.groupby('function')
sums = groups.sum()
fig1 = px.pie(sums, values='time', names=sums.index)
fig1.update_traces(textposition='inside', textinfo='percent+label')

fig2 = go.Figure()
for name, group in groups:
    fig2.add_trace(go.Scatter(x=group['step'], y=group['time'], mode='lines', name=str(name)))
fig2.update_layout(xaxis_title='Step', yaxis_title='Time')

df['cumulative_time'] = df.groupby('function')['time'].cumsum()
fig3 = px.line(df, x='step', y='cumulative_time', color='function')
fig3.update_layout(title='Cumulative Lines of Time per Steps', xaxis_title='Steps', yaxis_title='Cumulative Time')

df['tech_cumulative_time'] = df.groupby('tech')['time'].cumsum()

fig4 = px.line(df, x='step', y='tech_cumulative_time', color='tech')
fig4.update_layout(title='Cumulative Lines of Time per Steps',
                  xaxis_title='Steps',
                  yaxis_title='Cumulative Time')

fig1.show()
fig3.show()
fig2.show()
fig4.show()

In [4]:
for step in set(df['step']):
    print(f"Step {step} took {df[df['step']==step]['time'].sum()/60:.2f} minutes")
print(f"total time is: {df['time'].sum()/(60*60):.2f} hours")

Step 0 took 2.39 minutes
Step 1 took 1.98 minutes
total time is: 0.07 hours


In [5]:
times = df.groupby('function')['time'].mean()
ratios = times / times.min()
print(f"Average time:")
fig5 = px.pie(times, values='time', names=times.index)
fig5.update_traces(textposition='inside', textinfo='percent+label')
fig5.show()
ratios.sort_values()

Average time:


function
lfs git    1.000000
s3         2.184407
pyxet      2.200294
lakefs     2.600693
lfs s3     2.814187
git-xet    2.995406
dvc        3.118448
Name: time, dtype: float64